<a href="https://colab.research.google.com/github/luisr96/Delivery-Routing-for-BreakfastFiesta/blob/main/Breakfast_Fiesta_Delivery_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports and defining the route algorithm

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import haversine_distances
from math import radians, degrees

from itertools import combinations

import random

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

home = {}
with open("FriendLatLong.txt") as f:
    home = dict([line.split() for line in f])
    home.update((k, radians(float(v))) for k, v in home.items())

In [ ]:
def shortest_route(df):
    '''Finds shortest route to all points given a distance matrix 
    where the first row/column is the starting location "home"'''

    route = ['home']

    # replace all zero distances with nan
    df = df.replace(0,np.nan)

    # append minimum value in home column (closest client from home) to a list
    closest_in_col = df[['home']].idxmin(axis=0)[0]
    route.append(closest_in_col)

    # replace that value with nan since it's just been visited
    df = df.replace(df['home'].min(),np.nan)

    try:

        # until there is only one location to go to
        while (1 in df.count().values) == False:

            # find minimum in row of previous column and add to route
            closest_in_row = df.loc[closest_in_col].idxmin(axis=1)
            route.append(closest_in_row)

            # replace value with nan
            df = df.replace(df.loc[closest_in_col].min(),np.nan)

            # set nan to all combinations of places already visited
            for a, b in list(combinations(route,2)):
                df = df.replace(df.at[a, b], np.nan)

            # find minimum in column of previous row value and add to route
            closest_in_col = df[[closest_in_row]].idxmin(axis=0)[0]
            route.append(closest_in_col)

            # replace that value with nan
            df = df.replace(df[closest_in_row].min(),np.nan)

            # set nan to all combinations of places already visited
            for a, b in list(combinations(route,2)):
                df = df.replace(df.at[a, b], np.nan)

    except:

        return route

    # add the last remaining location (the only non-1 value)
    route.append(df.count()[df.count() != 1].index[0])

    return route

Getting data from the Google sheets

In [ ]:

# Google sheets
sheet_url = "https://docs.google.com/spreadsheets/d/1ssR3tbF8EcWUF7cVjfO8c2dNBYcU6rReQfvD1muv66A/edit#gid=2029231388"

# read in the Google Sheets file into a pandas dataframe
wb = gc.open_by_url(sheet_url)
sheet = wb.worksheet('Delivery Order Log')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.set_index("Client", inplace = True)
df = df[["Latitude", "Longitude"]]
df["Latitude"] = pd.to_numeric(df["Latitude"])
df["Longitude"] = pd.to_numeric(df["Longitude"])
df

,Latitude,Longitude
Client,,
Client 1,25.732334,-80.447624
Client 2,25.835857,-80.247880
Client 3,25.767807,-80.242130
Client 4,25.830913,-80.290649
Client 5,25.899364,-80.247531
Client 6,25.682746,-80.467798
Client 7,25.768700,-80.246811
Client 8,25.948150,-80.119947
Client 9,25.881422,-80.331037


Visualizing the geodata

In [ ]:
import folium

# create map
m = folium.Map(
    location=[degrees(home["lat"]), degrees(home["long"])],
    tiles='OpenStreetMap',
    zoom_start=9.8
)

# add client points
for index, row in df.iterrows():
    folium.Marker([row["Latitude"], row["Longitude"]], popup=row.name).add_to(m)

# add fit bounds
sw = df[['Latitude', 'Longitude']].min().values.tolist()
ne = df[['Latitude', 'Longitude']].max().values.tolist()
m.fit_bounds([sw, ne])

m

## Finding shortest distance

Creating Distance Matrix

In [ ]:
# convert latitude and longitudes to radians and add to lists
lats_in_radians = [radians(_) for _ in df["Latitude"]]
longs_in_radians = [radians(_) for _ in df["Longitude"]]

# combine them into a single latlong list
latlongs_in_radians = list(zip(lats_in_radians, longs_in_radians))

# insert home lat and long at the beginning of the latlong list
latlongs_in_radians.insert(0,(home['lat'],home['long']))

# create matrix using sklearn's haversine_distances
matrix = haversine_distances(latlongs_in_radians)

# multiply by earth's radius in miles to convert matrix to miles
distance_matrix = matrix * 3958.8

# make the matrix's index and column names go from home to all clients
distance_matrix = pd.DataFrame(distance_matrix, columns=['home'] + [f"Client {i}" for i in range(1,len(df.index)+1)],
                         index=['home'] + [f"Client {i}" for i in range(1,len(df.index)+1)])

distance_matrix

,home,Client 1,Client 2,Client 3,Client 4,Client 5,Client 6,Client 7,Client 8,Client 9,Client 10,Client 11,Client 12,Client 13,Client 14,Client 15,Client 16,Client 17,Client 18,Client 19,Client 20,Client 21,Client 22
home,0.000000,15.381242,1.856112,6.565101,3.481773,2.532365,18.520198,6.495850,9.869740,5.369156,28.127470,7.530090,7.136826,8.311297,5.797953,14.306726,5.013041,6.855764,2.991672,13.610595,2.362263,15.321447,7.736141
Client 1,15.381242,0.000000,14.338627,13.021274,11.906983,16.972883,3.649160,12.747237,25.250148,12.597828,15.261263,22.560224,13.891338,7.723017,16.667391,5.791268,15.355868,15.014190,12.389578,1.947130,13.331122,0.838223,16.143140
Client 2,1.856112,14.338627,0.000000,4.715418,2.681637,4.387991,17.297357,4.640628,11.110114,6.053487,26.530024,9.163909,8.159273,7.830868,4.523336,12.747817,6.428418,8.107382,2.383552,12.490707,2.751127,14.194184,6.167460
Client 3,6.565101,13.021274,4.715418,0.000000,5.302988,9.095975,15.226896,0.297745,14.593818,9.602061,22.954525,13.425262,11.954969,8.934555,3.838088,9.674713,10.797719,12.210490,5.528771,11.075501,6.759914,12.614581,3.230170
Client 4,3.481773,11.906983,2.681637,5.302988,0.000000,5.436419,15.044336,5.090625,13.349336,4.299454,24.883412,10.893609,6.654450,5.149233,6.717657,11.067861,5.778053,6.973687,0.522230,10.128829,1.891737,11.839699,7.797245
Client 5,2.532365,16.972883,4.387991,9.095975,5.436419,0.000000,20.292516,9.028201,8.615142,5.336651,30.318537,5.587341,6.304845,9.464846,7.951788,16.498237,3.687894,5.633509,4.914812,15.317900,3.691125,17.013125,10.062997
Client 6,18.520198,3.649160,17.297357,15.226896,15.044336,20.292516,0.000000,14.982802,28.361707,16.150483,11.730349,25.865382,17.522788,11.264265,19.028677,6.145330,18.901643,18.635222,15.549457,4.984779,16.610413,3.334190,18.132241
Client 7,6.495850,12.747237,4.640628,0.297745,5.090625,9.028201,14.982802,0.000000,14.695333,9.386146,22.815871,13.450968,11.745074,8.644233,4.059462,9.474431,10.638157,12.022040,5.335522,10.800848,6.593337,12.347750,3.526343
Client 8,9.869740,25.250148,11.110114,14.593818,13.349336,8.615142,28.361707,14.695333,0.000000,13.905109,37.445602,3.851525,14.383593,17.988462,11.316138,23.779329,11.727776,13.399967,12.861162,23.475911,11.999140,25.186897,13.518961
Client 9,5.369156,12.597828,6.053487,9.602061,4.299454,5.336651,16.150483,9.386146,13.905109,0.000000,27.052771,10.566521,2.375918,4.886298,10.565940,13.627658,2.758520,2.949359,4.131665,11.227005,3.305637,12.817140,11.970093


Shortest route

In [ ]:
shortest_route = shortest_route(distance_matrix)

df_route = df.reindex(shortest_route[1:])
df_route = df_route.dropna()
df_route

,Latitude,Longitude
Client,,
Client 2,25.835857,-80.247880
Client 18,25.837309,-80.286174
Client 4,25.830913,-80.290649
Client 20,25.857784,-80.284810
Client 9,25.881422,-80.331037
Client 12,25.912257,-80.347957
Client 17,25.924029,-80.333931
Client 16,25.913574,-80.304727
Client 5,25.899364,-80.247531


Showing a route line

In [ ]:
points = []
for index, row in df_route.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    point = (lat, long)
    points.append(point)

folium.PolyLine(points, color="green", weight=5, opacity=1).add_to(m)
m

## Job scheduling for n drivers

Distance from last point to current point

In [ ]:
miles_to_get_there = []
total = 0

# get a list of each distance
for previous, current in zip(shortest_route, shortest_route[1:]):
    try:
        miles_to_get_there.append(distance_matrix.at[previous, current])
        total += distance_matrix.at[previous, current]
    except:
        miles_to_get_there.append(0)

# insert 0 at the beginning of list
miles_to_get_there.insert(0,0)

# combine into a list of tuples: place and distance to get there
place_and_miles_to_get_there = list(zip(shortest_route, miles_to_get_there))
place_and_miles_to_get_there.pop(0)

print(place_and_miles_to_get_there)
print(f"\nTotal distance: {total:.2f} miles" )

[('Client 2', 1.856111892432243), ('Client 18', 2.383551665397463), ('Client 4', 0.5222304906803718), ('Client 20', 1.891736776183288), ('Client 9', 3.3056371273223126), ('Client 12', 2.375918448641937), ('Client 17', 1.19223913388424), ('Client 16', 1.9532868916073098), ('Client 5', 3.6878940988634334), ('Client 11', 5.587341480446928), ('Client 8', 3.8515251561880017), ('Client 14', 11.316138007609654), ('Client 22', 2.3313996380943314), ('Client 3', 3.2301700346803757), ('Client 7', 0.29774506891879016), ('Client 13', 8.644233112665754), ('Client 19', 6.357665551627981), ('Client 21', 1.7109928055036), ('Client 1', 0.8382232201551643), ('Client 6', 3.649160132174134), ('Client 15', 6.145329507321997), ('Client 10', 13.826730865062498)]

Total distance: 86.96 miles


Dividing work to n drivers

In [ ]:
print(f"Job sequencing for n drivers\n")

driver_dict = {}
sum_ = 0
i = 1

n = 3

for place in place_and_miles_to_get_there:    
    sum_ += place[1]
    # when sum + item > total, reset sum and go to next key
    if sum_ + place[1] > (total/n):
        sum_ = 0
        i += 1
    driver_dict.setdefault(f"Driver {i}", []).append(place)

driver_dict

Job sequencing for n drivers



{'Driver 1': [('Client 2', 1.856111892432243),
  ('Client 18', 2.383551665397463),
  ('Client 4', 0.5222304906803718),
  ('Client 20', 1.891736776183288),
  ('Client 9', 3.3056371273223126),
  ('Client 12', 2.375918448641937),
  ('Client 17', 1.19223913388424),
  ('Client 16', 1.9532868916073098),
  ('Client 5', 3.6878940988634334)],
 'Driver 2': [('Client 11', 5.587341480446928),
  ('Client 8', 3.8515251561880017),
  ('Client 14', 11.316138007609654),
  ('Client 22', 2.3313996380943314),
  ('Client 3', 3.2301700346803757),
  ('Client 7', 0.29774506891879016)],
 'Driver 3': [('Client 13', 8.644233112665754),
  ('Client 19', 6.357665551627981),
  ('Client 21', 1.7109928055036),
  ('Client 1', 0.8382232201551643),
  ('Client 6', 3.649160132174134),
  ('Client 15', 6.145329507321997)],
 'Driver 4': [('Client 10', 13.826730865062498)]}

In [ ]:
# create map
m_multiple_drivers = folium.Map(
    location=[degrees(home["lat"]), degrees(home["long"])],
    tiles='stamentoner',
    zoom_start=9.8
)

# add client points
for index, row in df.iterrows():
    folium.Marker([row["Latitude"], row["Longitude"]], popup=row.name).add_to(m_multiple_drivers)

# add fit bounds
sw = df[['Latitude', 'Longitude']].min().values.tolist()
ne = df[['Latitude', 'Longitude']].max().values.tolist()
m_multiple_drivers.fit_bounds([sw, ne])

In [ ]:
colors = iter(['red', 'blue', 'green', 'purple', 'orange', 'pink', 'grey',
               'black', 'lightblue', 'lightgreen'])

for route in driver_dict.values():
    driver_routes = []

    for client in route:
        coord = (df.loc[df.index == client[0], 'Latitude'].iloc[0],
                             df.loc[df.index == client[0], 'Longitude'].iloc[0])

        driver_routes.append(coord)
    
    # start and end at home
    driver_routes.insert(0, (degrees(home['lat']),degrees(home['long'])))
    driver_routes.append((degrees(home['lat']),degrees(home['long'])))

    folium.PolyLine(driver_routes, color=next(colors), weight=5, opacity=1).add_to(m_multiple_drivers)

m_multiple_drivers.save(f"{len(driver_dict)}_drivers_plan.html")
m_multiple_drivers

[(25.86271384772938, -80.24721329997537), (25.835857, -80.24787950000001), (25.837309100000002, -80.286174), (25.8309134, -80.29064890000001), (25.8577835, -80.2848099), (25.881422, -80.3310372), (25.9122565, -80.3479573), (25.924028699999997, -80.3339306), (25.9135743, -80.3047273), (25.8993637, -80.2475311), (25.86271384772938, -80.24721329997537)]
[(25.86271384772938, -80.24721329997537), (25.9543107, -80.1815611), (25.948150100000003, -80.11994659999999), (25.796804, -80.18951170000001), (25.763073000000002, -80.19048509999999), (25.7678073, -80.2421295), (25.7687, -80.2468108), (25.86271384772938, -80.24721329997537)]
[(25.86271384772938, -80.24721329997537), (25.821505300000002, -80.372781), (25.7386062, -80.4171248), (25.723253699999997, -80.4386933), (25.732333800000003, -80.447624), (25.682746100000003, -80.4677982), (25.6872099, -80.3692293), (25.86271384772938, -80.24721329997537)]
[(25.86271384772938, -80.24721329997537), (25.5132707, -80.4789499), (25.86271384772938, -80.2